# Automatic Record Merge in HubSpot

Nosotros realizamos un merge automatico de registros en HubSpot mediante la implementación de la herramienta de "Merge records" de HubSpot, por lo cual el merge se rige bajo las siguientes normas:

**Doc:** https://knowledge.hubspot.com/crm-setup/merge-records

Esta implementación de la herramienta de HubSpot la realizamo a traves de la API.

Este codigo trabaja haciendo uso del "Key" de los duplicados que obtenemos despues de implementar las funciones de encontrar duplicados.

# Set environment variables

In [92]:
import os

In [93]:
os.environ["path"] = r"C:\Users\Juan Pablo Pelaez\Documents\PyDev-Operational-Library-For-Data-Engineers"
os.environ["access_token"] = 'pat-na1-3fa39b1a-5a4d-4161-a611-8d99019072b9'
os.environ["records_object_type"] = 'contacts'
os.environ["file_path"] = r"C:\Users\Juan Pablo Pelaez\Documents\Training\Training APIs"
os.environ["file_name"] = 'Duplicate Contact Records Found.xlsx'

## Libraries

In [94]:
import requests
import json

import pandas as pd
import numpy as np

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

from IPython.display import display, HTML, clear_output

#### Data team library

Your library path

In [95]:
path = os.getenv('path')

In [96]:
import sys
sys.path.insert(0,path)

from functions.data_transformations.delete_unnecessary_blank_spaces import delete_unnecessary_blank_spaces

## Parameters - Input Values

#### Access API

In [97]:
access_token = os.getenv('access_token') # Input

headers = {'Content-Type': 'application/json',
            'authorization': 'Bearer {}'.format(access_token)}

#### Info Records

In [98]:
records_object_type = os.getenv('records_object_type')

#### Frame

In [99]:
file_path = os.getenv('file_path')

In [100]:
file_path

'C:\\Users\\Juan Pablo Pelaez\\Documents\\Training\\Training APIs'

In [101]:
file_name = os.getenv('file_name')

In [102]:
file_name

'Duplicate Contact Records Found.xlsx'

In [103]:
df = pd.read_excel(file_path+'\\'+file_name)

In [104]:
df = df.replace(np.nan, '')

In [105]:
df

,Record ID,First Name,Last Name,Phone Number,Email,original mail,Create Date,Last Activity Date
0,102020159252,Tyson,Newman,(+353) 8350 2738,tyson_newman1021718471@yahoo.com,tyson_newman1021718471+103@yahoo.com,2025-02-25 07:58,
1,102021803047,,,(+353) 8350 2738,tyson_newman1021718471@yahoo.com,tyson_newman1021718471+101@yahoo.com,2025-02-25 07:58,
2,102033748778,,,,tyson_newman1021718471@yahoo.com,tyson_newman1021718471@yahoo.com,2025-02-25 07:58,
3,102023206209,,,(+353) 8350 2738,tyson_newman1021718471@yahoo.com,tyson_newman1021718471+102@yahoo.com,2025-02-25 07:58,
4,102033410128,,,(+44) 5172 2704,sebastian_parker1458479109@joiniaa.com,sebastian_parker1458479109+101@joiniaa.com,2025-02-25 07:58,
...,...,...,...,...,...,...,...,...
274,102021803049,Alan,Warner,(+44) 2602 8217,alan_warner22954315@fuliss.net,alan_warner22954315+101@fuliss.net,2025-02-25 07:58,
275,102021344113,Alan,Warner,(+44) 2602 8217,alan_warner22954315@fuliss.net,alan_warner22954315@fuliss.net,2025-02-25 07:58,
276,102021803051,,,,alan_warner22954315@fuliss.net,alan_warner22954315+104@fuliss.net,2025-02-25 07:58,
277,102020822178,,,(+44) 2602 8217,alan_warner22954315@fuliss.net,alan_warner22954315+103@fuliss.net,2025-02-25 07:58,


In [106]:
printmd("<h3><span style='color:blue'>You will work with {} records</span></h3>".format(len(df)))

<h3><span style='color:blue'>You will work with 279 records</span></h3>

In [107]:
df.head(5)

,Record ID,First Name,Last Name,Phone Number,Email,original mail,Create Date,Last Activity Date
0,102020159252,Tyson,Newman,(+353) 8350 2738,tyson_newman1021718471@yahoo.com,tyson_newman1021718471+103@yahoo.com,2025-02-25 07:58,
1,102021803047,,,(+353) 8350 2738,tyson_newman1021718471@yahoo.com,tyson_newman1021718471+101@yahoo.com,2025-02-25 07:58,
2,102033748778,,,,tyson_newman1021718471@yahoo.com,tyson_newman1021718471@yahoo.com,2025-02-25 07:58,
3,102023206209,,,(+353) 8350 2738,tyson_newman1021718471@yahoo.com,tyson_newman1021718471+102@yahoo.com,2025-02-25 07:58,
4,102033410128,,,(+44) 5172 2704,sebastian_parker1458479109@joiniaa.com,sebastian_parker1458479109+101@joiniaa.com,2025-02-25 07:58,


## Empty Keys

Vamos a realizar una pequeña revisión de las Keys, no debe existir ninguna key que este en blanco, realizar un merge de registros con keys vacias podria incluso eliminar bases de datos completas.

In [108]:
df['Key'] = df['Email'].apply(delete_unnecessary_blank_spaces)

In [109]:
detect_empty_keys = df.loc[df['Key'] == '']

if len(detect_empty_keys) != 0:
    printmd("<h3><span style='color:red'>There are keys configured as empty</span></h3> please check these keys that are empty [''], otherwise you will end up making merges that should not be made.")
else:
     printmd("<h3><span style='color:green'>Keys are properly configured</span></h3> Go ahead!")

<h3><span style='color:green'>Keys are properly configured</span></h3> Go ahead!

In [110]:
detect_empty_keys

,Record ID,First Name,Last Name,Phone Number,Email,original mail,Create Date,Last Activity Date,Key


## Define number of duplicates

En esta longitud de valores seremos capaces de recorrer todo el frame de duplicados, obteniendo todas las key unicas que nos señalan los duplicados.

In [111]:
duplicates = df.drop_duplicates(subset=['Key'])
duplicates = duplicates.reset_index(drop=True)

In [112]:
duplicates

,Record ID,First Name,Last Name,Phone Number,Email,original mail,Create Date,Last Activity Date,Key
0,102020159252,Tyson,Newman,(+353) 8350 2738,tyson_newman1021718471@yahoo.com,tyson_newman1021718471+103@yahoo.com,2025-02-25 07:58,,tyson_newman1021718471@yahoo.com
1,102033410128,,,(+44) 5172 2704,sebastian_parker1458479109@joiniaa.com,sebastian_parker1458479109+101@joiniaa.com,2025-02-25 07:58,,sebastian_parker1458479109@joiniaa.com
2,102032514367,,,,sebastian_hope1518365261@hourpy.biz,sebastian_hope1518365261+102@hourpy.biz,2025-02-25 07:58,,sebastian_hope1518365261@hourpy.biz
3,102033492005,Ryan,Welsch,(+44) 7786 3560,ryan_welsch107397896@famism.biz,ryan_welsch107397896+103@famism.biz,2025-02-25 07:58,,ryan_welsch107397896@famism.biz
4,102020032396,,,,ryan_upsdell175534425@vetan.org,ryan_upsdell175534425+105@vetan.org,2025-02-25 07:58,,ryan_upsdell175534425@vetan.org
5,102033185298,Roger,Jones,,roger_jones1323472465@gompie.com,roger_jones1323472465@gompie.com,2025-02-25 07:58,,roger_jones1323472465@gompie.com
6,102033030275,Rocco,Palmer,(+44) 8134 0522,rocco_palmer160419466@infotech44.tech,rocco_palmer160419466+106@infotech44.tech,2025-02-25 07:58,,rocco_palmer160419466@infotech44.tech
7,102031826464,,,,rocco_martin507925367@eirey.tech,rocco_martin507925367+103@eirey.tech,2025-02-25 07:58,,rocco_martin507925367@eirey.tech
8,102020822173,,,,ramon_preston2096761104@sheye.org,ramon_preston2096761104+104@sheye.org,2025-02-25 07:58,,ramon_preston2096761104@sheye.org
9,102033395078,Phillip,Cooper,(+44) 2026 4200,phillip_cooper2140654386@jiman.org,phillip_cooper2140654386+104@jiman.org,2025-02-25 07:58,,phillip_cooper2140654386@jiman.org


In [113]:
df.head(4)

,Record ID,First Name,Last Name,Phone Number,Email,original mail,Create Date,Last Activity Date,Key
0,102020159252,Tyson,Newman,(+353) 8350 2738,tyson_newman1021718471@yahoo.com,tyson_newman1021718471+103@yahoo.com,2025-02-25 07:58,,tyson_newman1021718471@yahoo.com
1,102021803047,,,(+353) 8350 2738,tyson_newman1021718471@yahoo.com,tyson_newman1021718471+101@yahoo.com,2025-02-25 07:58,,tyson_newman1021718471@yahoo.com
2,102033748778,,,,tyson_newman1021718471@yahoo.com,tyson_newman1021718471@yahoo.com,2025-02-25 07:58,,tyson_newman1021718471@yahoo.com
3,102023206209,,,(+353) 8350 2738,tyson_newman1021718471@yahoo.com,tyson_newman1021718471+102@yahoo.com,2025-02-25 07:58,,tyson_newman1021718471@yahoo.com


## Merge Records

In [114]:
len('hjgkljyugluygloyug')

18

In [118]:
for i in range(49,len(duplicates)):
    
    clear_output(wait=True)#borrar la salida de la celda, para poder imprimir los detalles mas recientes
    
    print('Loop # {} of {}'.format(i, len(duplicates)-1))# aplicar fotmaro en el primer {i} of {duplicates} y el -1 para obtener el ultimo dato 
    
    ## Our key to detect duplicates
    key = duplicates.at[i, 'Key'] #la variable key va almacenar la llave en la fila i, y la columna key 
   
    ## Find Duplicates Frame
    mini = df.loc[(df['Key'] == key)] # guardar en mini  la columna key pero todo lo de la variable key 
    
    ## To select the newest or oldest record as primary_record_id [ascending=True/False]
    #mini = mini.sort_values(by='Create Date', ascending=True) 
    #mini['es_primario'] = ~mini['original mail'].str.contains(r'\+')

    mini['tamEmail']=mini['original mail'].apply(lambda x:len(x))
    
    mini = mini.sort_values(by=['tamEmail'] ,ascending=True)
    mini = mini.reset_index(drop=True)#dejar el index 'predeterminad' o el de defecto
    
    # # # Loop for merging
    primary_record_id = int(mini.at[0, 'Record ID']) # Start record # trae el el record id de mini en entero desde la posicion 0 y dela columna record id 
    
    for j in range(1, len(mini)): #Start in 1 because primary_record_id will be the first [i = 0] in the start
        
        to_merge_record_id = int(mini.at[j, 'Record ID'])# guarda en la variable un entero de cada recorrido en la columna record id 
        
        payload = json.dumps({'primaryObjectId': primary_record_id,
                              'objectIdToMerge': to_merge_record_id})#combierte el las variables a un formato json y toma el record id y lo mergea con el rrecorrido echo
        
        url = 'https://api.hubapi.com/crm/v3/objects/{}/merge'.format(records_object_type)#es un formato para decirle a la url que ahi va lo que el usuario quiera 
        
        api_response = requests.request("POST", url, data=payload, headers=headers) # guardamos lo que responda el llamado 
        
        ## El resultado del merge es un nuevo registro con un nuevo ID, este se convierte en el primary_record_id
        primary_record_id = api_response.json()['id']
        
        print('Merge # {} - Merging {} into {}'.format(j, to_merge_record_id, primary_record_id))
        print(api_response)
        
     ## Puse este break-loop_i para que revise el primer resultado que le dio - recuerde que desmerge no hay

Loop # 49 of 49


C:\Users\Juan Pablo Pelaez\AppData\Local\Temp\ipykernel_10108\2686015421.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini['tamEmail']=mini['original mail'].apply(lambda x:len(x))


Merge # 1 - Merging 102021803049 into 102064929568
<Response [200]>
Merge # 2 - Merging 102021803051 into 102089672753
<Response [200]>
Merge # 3 - Merging 102020822178 into 102055692120
<Response [200]>
Merge # 4 - Merging 102033443462 into 102082956606
<Response [200]>
